In [ ]:
from ipywidgets import Layout, HBox, VBox, FloatText
import copy
import json
import pygeohash as gh
from ipyleaflet import (
    Map, basemaps, TileLayer, SplitMapControl, GeoJSON, Polygon, DrawControl, LayersControl, Layer, LayerGroup
)

In [ ]:
m = Map(center=[53.88, 27.45], zoom=3)

# geojson layer with hover handler
with open('./europe_110.geo.json') as f:
    data = json.load(f)

def decorate_geojson_for_display(data):
    disp_data = copy.deepcopy(data)
    for feature in disp_data['features']:
        feature['properties']['style'] = {
            'color': 'grey',
            'weight': 1,
            'fillColor': 'grey',
            'fillOpacity': 0.2
        }
        feature_id = feature['properties']['iso_a3']
    return disp_data
        
def create_lookup_dictionary_id_geometry(data):
    geometry_dict = {}
    for feature in data['features']:
        feature_id = feature['properties']['iso_a3']
        geometry_dict[feature_id] = feature['geometry']
    return geometry_dict

        
def click_handler_selected(event=None, properties=None, id=None, **args):
    print('Selected: '  + str(event))

def click_handler_geojson(event=None, properties=None, id=None, **args):
    print('GeoJSON: '  + str(event))
    global lookup_dict
    global selected_dict
    global selected_layer
    global layer_geojson
    if properties is not None:
        feature_id = properties['iso_a3']
        if lookup_dict[feature_id]['type'] == 'Polygon':
            if feature_id in selected_dict:
                rem_polygon = selected_dict.pop(feature_id)
                selected_layer.remove_layer(rem_polygon)
            else:
                feature = lookup_dict[feature_id]
                polygon = Polygon(locations=[[p[1], p[0]] for p in feature['coordinates'][0]], color='yellow', fillColor='yellow')
                selected_dict[feature_id] = polygon
                selected_layer.add_layer(polygon)
    if len(args.keys()) > 0:
        lat = args['coordinates'][0]
        lon = args['coordinates'][1]

def print_anything(event=None, properties=None, id=None):
    print("Anything")
        
disp_data = decorate_geojson_for_display(data)
lookup_dict = create_lookup_dictionary_id_geometry(disp_data)
selected_dict = {}

layer_geojson = GeoJSON(data=disp_data, hover_style={'fillColor': 'red'}, name='GeoJson')
m.add_layer(layer_geojson)
layer_geojson.on_click(click_handler_geojson)

selected_layer = LayerGroup(name='Selected')
selected_layer.on_click(print_anything)
m.add_layer(selected_layer)

m.add_control(DrawControl())
lc = LayersControl()
m.add_control(lc)
m

In [ ]:

layer_geojson.send({'msg':'change:style'})

In [ ]:
help(layer_geojson.send)